# Plotting Figures for the BIDS BEP_006 EEG Report

In [ ]:
import os

from tqdm import tqdm
import metapub
import pandas as pd
import numpy as np

%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

# Get PubMed search results

In [ ]:
# Use metapub package to search PubMed
fetch = metapub.PubMedFetcher()

# Our query and range of years
query = 'EEG[Title/Abstract]'
years = range(1929, 2018)

In [ ]:
# For each year, get all articles and save
data = list()
for year in tqdm(years):
    pmids = fetch.pmids_for_query(query, since=year, until=year, retmax=int(1e6))
    n_articles = len(pmids)
    data.append([year, n_articles])

In [ ]:
# Format as data frame
df = pd.DataFrame(data, columns=['year', 'count'])

In [ ]:
# We are concerned from the year in which the first publication is indexed 
row_idx = np.min(np.where(df['count'] > 0)[0])
data = df.loc[row_idx::, :].copy()

# Which year was it?
display(data['year'].min())

## Figure showing PubMed search results

In [ ]:
# Set the context and start a new figure
sns.set_style('ticks')
sns.set_context('paper', font_scale=2.)

fig, ax = plt.subplots(1, 1, figsize=(6.5, 3))

# Plot data and regression fit
sns.regplot(x='year', y='count', data=data, ax=ax,
            fit_reg=True, order=5, ci=None, truncate=False,
            scatter_kws={'color': 'cornflowerblue'},
            line_kws={'color': 'crimson'})

# Labels
ax.set_xlabel('Year')
ax.set_ylabel('Count')
#ax.set_title('Number of Publications Containing "EEG" in Title or Abstract')

# Get rid of spines
sns.despine()

# Crop the x axis to next round year
assert data['year'].min() == 1947
ax.set_xlim((1950, None))

# Save figure
fname = 'fig_pubmed_results.png'
fpath = os.path.join(os.getcwd(), 'figures', fname)
fig.savefig(fpath, dpi=600, bbox_inches='tight')